# WDRC Test

In this example, a test signal is injected into the magnitude stage,
bypassing the filter bank and hilbert transform stages.
This example is useful for testing WDRC, in particular the
attack/release times.

This verifies and reproduces the Matlab test script main_wdrc_test.m

In [1]:
# make Jupyter use the whole width of the browser
from IPython.display import Image, display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('..')

In [3]:
import numpy as np
from scipy.io import loadmat, savemat
from rtmha.elevenband import Wdrc11
import plotly.graph_objects as go

In [4]:
# get the results from main_wdrc_test.m
res = loadmat('../test/wdrc_test_out.mat')

In [5]:
step = res['step'].T

In [6]:
g50 = 30*np.ones(11)
g80 = 10*np.ones(11)
kneelow = 45*np.ones(11)
mpo = 100*np.ones(11)
AT = 5*np.ones(11)
RT = 50*np.ones(11)
rate = np.array([1/16, 1/16, 1/16, 1/8, 1/8, 1/4, 1/4, 1/2, 1/2, 1, 1])
fs = 32000
L = 350
cal_mic = np.zeros(11)
cal_rec = np.zeros(11)

In [7]:
w = Wdrc11(g50, g80, kneelow, mpo, AT, RT, cal_mic, cal_rec, len(step[10][0][0]), True, True)

In [ ]:
w.get_param()

In [8]:
m, b, c, kneeup, alpha1, alpha2 = w.get_param_test()

In [9]:
# check alpha1 for matlab and c++
np.allclose(alpha1, res['alpha1'][0], rtol=1e-4)

False

In [10]:
# check alpha2 for matlab and c++
np.allclose(alpha2, res['alpha2'][0], rtol=1e-4)

False

In [11]:
# verify other parameters are the same
print(np.allclose(b, res['b'][0], rtol=1e-4))
print(np.allclose(c, res['c'][0], rtol=1e-4))
print(np.allclose(m, res['m'][0], rtol=1e-4))
print(np.allclose(kneeup, res['kneeup'][0], rtol=1e-4))

True
True
False
False


In [12]:
max(step[10][0][0])

31622.776601683792

In [13]:
20*np.log10(31622.776601683792)

90.0

If the above all passed then WDRC init is OK.

In [14]:
interactive=True

for i in range(11):
    B1 = w.B1[i]
    B2 = w.B2[i]
    mag = step[i][0][0]
    w.set_A(i, B1 - 55)
    gain = w._agc(mag, w.A[i], kneelow[i], kneeup[i], 100.0, m[i], b[i], c[i], alpha1[i], alpha2[i])

    fig = go.Figure()
    # plot((0:1000/(fs*rate(i)):L-1000/(fs*rate(i))), 20*log10(magnitude.*gain),'r', 'linewidth', 2)
    period = 1000/(fs*rate[i])
    x = np.arange(0, L, period)
    y = 20*np.log10(mag*gain)
    fig.add_trace(go.Scatter(x=x, 
                        y=y,
                        line={'color':'red'},
                        opacity=.8,         
                        name='Output Magnitude'))
    fig.add_trace(go.Scatter(x=x, 
                        y=20*np.log10(mag),
                        line={'color':'blue'},
                        opacity=.8,         
                        name='Input Magnitude'))
    fig.add_hline(y=B2+3, line_dash="solid", line_color="green", opacity=.5, annotation_text="+3dB of Desired Level")
    fig.add_hline(y=B1-4, line_dash="solid", line_color="magenta", opacity=.5, annotation_text="-4dB of Desired Level")
    fig.add_hline(y=B1, line_dash="dash", line_color="black", opacity=.5, annotation_text="Desired Level")
    fig.add_hline(y=B2, line_dash="dash", line_color="black", opacity=.5, annotation_text="Desired Level")
    fig.add_vline(x=50+AT[i], line_dash="solid", line_color="green", opacity=.5)
    fig.add_vline(x=100+RT[i], line_dash="solid", line_color="magenta", opacity=.5)
    fig.update_layout(title=f'Verifying Attack/Release Time: Band={i} AT={AT[i]}, RT={RT[i]}',
                        xaxis_title='Time(ms)',
                        yaxis_title='Log Mag (dB)',
                        template='plotly_white')
    if interactive:
        fig.show()        
    else:
        fig.write_image(f"wdrc{i}.png", engine="kaleido")
        display(Image(f"wdrc{i}.png", width=4096))


TypeError: _agc() takes exactly 12 positional arguments (10 given)